In [1]:
import os 
os.chdir('../../')
os.environ["DPM_TQDM"] = "False"

In [2]:
from backbones.sana import SANA

# 사용 예
model = SANA(model_id='Efficient-Large-Model/Sana_600M_1024px_diffusers',
             trainable=True)
print(model)

/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  5.85it/s]


In [3]:
from solvers.euler_solver import Euler_Solver

pos_texts = ["A serene twilight view of a traditional Korean hanok village nestled between misty mountain slopes: curved midnight-blue tiled eaves, softly glowing paper lanterns swaying in the breeze; ancient pine trees arching over stone pathways; intricate wooden lattice windows casting delicate shadows; a lone scholar in flowing hanbok practicing calligraphy beside a koi pond with lotus petals drifting on the water; cinematic 8K ultra-realism with dynamic volumetric moonlight filtering through morning mist; painterly strokes blending classical Joseon-era ink wash with modern hyperrealism; shot on RED Monstro 8K, 50 mm f/1.2 lens; subtle film grain; maximum fidelity; emotional atmosphere."]
neg_texts = ["lowres, bad anatomy, deformed, blurry, pixelated, oversaturated, underexposed, overexposed, artifact, jpeg artifacts, watermark, text, logo, extra limbs, mutated hands, unnatural colors, noisy background, out of focus, poor composition, cultural clichés, stereotype exaggeration, flat lighting, glitch"]

model_fn, noise_schedule, latents = model.get_model_fn(pos_conds=pos_texts, neg_conds=neg_texts, guidance_scale=4.5, seeds=[42])
print('done')

done


In [4]:
import torch
import torch.nn.functional as F
from torch import amp
from solvers.euler_solver import Euler_Solver

# with torch.no_grad():
#     solver = Euler_Solver(model_fn, noise_schedule, algorithm_type="data_prediction")
#     euler_latents = solver.sample(latents, steps=10, skip_type='time_uniform_flow', flow_shift=3.0)
#     euler_samples = model.decode_vae(euler_latents['samples'])
    
# euler_samples[0]

In [5]:
import torch
from torch import amp
from solvers.dual.static.gdual_box_solver import GDual_Box_Solver

device = 'cuda:0'
solver = GDual_Box_Solver(model_fn, noise_schedule, steps=5, skip_type="time_uniform_flow", flow_shift=3.0, param_dim=(1, 1, 1))
solver = solver.to(device)

    
optimizer = torch.optim.AdamW(solver.parameters(), lr=1e-3)
scaler = amp.GradScaler()
print('done')

#gdual_samples[0]

done


In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

pbar = tqdm(range(1000))
for iter in pbar:
    optimizer.zero_grad(set_to_none=True)
    with amp.autocast(device_type=device, dtype=torch.bfloat16):
        preds = solver.sample(latents)
        loss = F.mse_loss(preds, torch.randn_like(preds))

        pbar.set_postfix({'loss': loss.item()})

    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    # if iter % 100 == 0:
    #     with torch.no_grad():
    #         pr_samples = model.decode_vae(pr_latents)
            
    #         import matplotlib.pyplot as plt
    #         plt.imshow(pr_samples[0])
    #         plt.show()

print('done')

  1%|          | 12/1000 [00:09<11:59,  1.37it/s, loss=2.36]